### 0. Install Dependencies

In [16]:
!nvcc --version
# For TensorFlow
!conda install tensorflow-gpu


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_19:00:59_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
^C


In [17]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))



Num GPUs Available:  0


In [18]:
!python --version


Python 3.9.12


In [19]:
!pip install --upgrade pip

In [20]:
!pip install gymnasium keras-rl2 gymnasium[atari]

In [2]:
import tensorflow as tf
print(tf.__version__)

2.9.1


### 1. Test Random Environment with OpenAi Gym

In [5]:
import gymnasium as gym
import random

In [6]:
env = gym.make('SpaceInvaders-v4', render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [7]:
#Actions tjhat agent can take
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [9]:
episodes = 5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    truncated = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, truncated, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:315.0
Episode:2 Score:100.0
Episode:3 Score:155.0
Episode:4 Score:165.0
Episode:5 Score:455.0


### 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Convolution3D, Convolution1D
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, Ftrl

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(16, (5,5), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(height, width, channels, actions)

In [9]:
# Model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 52, 39, 16)     1216      
                                                                 
 flatten (Flatten)           (None, 97344)             0         
                                                                 
 dense (Dense)               (None, 6)                 584070    
                                                                 
Total params: 585,286
Trainable params: 585,286
Non-trainable params: 0
_________________________________________________________________


### 3. Build Agent with Keras-RL


In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [2]:
def build_agent(model,actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.,value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [3]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-3))
dqn.fit(env, nb_steps=2000, visualize=False, verbose=2)

NameError: name 'model' is not defined